# Como combinar dados de várias tabelas

Iniciando os pacotes e lendo os arquivos

In [5]:
import pandas as pd

air_quality_no2 = pd.read_csv("../data/air_quality_no2_long.csv", parse_dates=True)
air_quality_no2 = air_quality_no2[["date.utc", "location", "parameter", "value"]]
air_quality_no2.head()

,date.utc,location,parameter,value
0,2019-06-21 00:00:00+00:00,FR04014,no2,20.0
1,2019-06-20 23:00:00+00:00,FR04014,no2,21.8
2,2019-06-20 22:00:00+00:00,FR04014,no2,26.5
3,2019-06-20 21:00:00+00:00,FR04014,no2,24.9
4,2019-06-20 20:00:00+00:00,FR04014,no2,21.4


In [ ]:
air_quality_pm25 = pd.read_csv("../data/air_quality_pm25_long.csv", parse_dates=True)
air_quality_pm25 = air_quality_pm25[["date.utc", "location", "parameter", "value"]]
air_quality_pm25.head()

,date.utc,location,parameter,value
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5


Concatenando objetos  

Quero combinar as medidas de NO2 e PM25, duas tabelas com estrutura semelhante, em uma única tabela.
A função concat(), executa operações de concatenação de várias tabelas ao longo de um dos eixos (linha ou coluna).

In [6]:
air_quality = pd.concat([air_quality_pm25, air_quality_no2], axis=0)
air_quality.head()

,date.utc,location,parameter,value
0,2019-06-18 06:00:00+00:00,BETR801,pm25,18.0
1,2019-06-17 08:00:00+00:00,BETR801,pm25,6.5
2,2019-06-17 07:00:00+00:00,BETR801,pm25,18.5
3,2019-06-17 06:00:00+00:00,BETR801,pm25,16.0
4,2019-06-17 05:00:00+00:00,BETR801,pm25,7.5


Por padrão, a concatenação é feita ao longo do eixo 0, de modo que a tabela resultante combina as linhas das tabelas de entrada.  
Vamos verificar o formato das tabelas original e concatenada para comprovar a operação:

In [11]:
print('Shape of the "air_quality_pm25" table: ', air_quality_pm25.shape)
print('Shape of the "air_quality_no2" table: ', air_quality_no2.shape)
print('Shape of the resulting "air_quality" table: ', air_quality.shape)

Shape of the "air_quality_pm25" table:  (1110, 4)
Shape of the "air_quality_no2" table:  (2068, 4)
Shape of the resulting "air_quality" table:  (3178, 4)


Portanto, a tabela resultante tem 3178 = 1110 + 2068 linhas.

O argumento axis retornará em diversos métodos pandas que podem ser aplicados ao longo de um eixo.  
O DataFrame possui dois eixos correspondentes:

* O primeiro, que se estende verticalmente para baixo, através das linhas (eixo 0)
* O segundo, que se estende horizontalmente, através das colunas (eixo 1).  

A maioria das operações, como concatenação ou estatísticas de resumo, são aplicadas, por padrão,  
entre linhas (eixo 0), mas também podem ser aplicadas entre colunas.

A classificação da tabela com base nas informações de data e hora ilustra também a combinação de ambas as tabelas,  
com a parametercoluna definindo a origem da tabela ( no2 da tabela air_quality_no2 ou pm25 da tabela air_quality_pm25):

In [12]:
air_quality = air_quality.sort_values("date.utc")
air_quality.head()

,date.utc,location,parameter,value
2067,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0
1003,2019-05-07 01:00:00+00:00,FR04014,no2,25.0
100,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5
1098,2019-05-07 01:00:00+00:00,BETR801,no2,50.5
1109,2019-05-07 01:00:00+00:00,London Westminster,pm25,8.0


Neste exemplo específico, a coluna *parameter*, fornecida pelos dados, garante que cada uma das tabelas originais possa ser identificada.  
Isso nem sempre acontece. A função *concat*, fornece uma solução conveniente com o argumento *keys*,  
adicionando um índice de linha (hierárquico) adicional.  
Por exemplo:

In [13]:
air_quality_ = pd.concat([air_quality_pm25, air_quality_no2], keys=["PM25", "NO2"])
air_quality_.head()

date.utc location parameter  value
PM25 0  2019-06-18 06:00:00+00:00  BETR801      pm25   18.0
     1  2019-06-17 08:00:00+00:00  BETR801      pm25    6.5
     2  2019-06-17 07:00:00+00:00  BETR801      pm25   18.5
     3  2019-06-17 06:00:00+00:00  BETR801      pm25   16.0
     4  2019-06-17 05:00:00+00:00  BETR801      pm25    7.5

A existência de múltiplos índices de linha/coluna simultaneamente não foi mencionada nestes tutoriais.  
A indexação hierárquica, ou MultiIndex, é um recurso avançado e poderoso do Pandas para analisar dados de dimensões superiores.

A multiindexação está fora do escopo desta introdução ao Pandas.  
Por enquanto, lembre-se de que a função reset_index pode ser usada para converter qualquer nível de um índice em uma coluna,  
por exemplo air_quality.reset_index(level=0)

# Junte tabelas usando um identificador comum

Adicione as coordenadas da estação, fornecidas pela tabela de metadados das estações, às linhas correspondentes na tabela de medições.

In [14]:
stations_coord = pd.read_csv("../data/air_quality_stations.csv")
stations_coord.head()

,location,coordinates.latitude,coordinates.longitude
0,BELAL01,51.23619,4.38522
1,BELHB23,51.17030,4.34100
2,BELLD01,51.10998,5.00486
3,BELLD02,51.12038,5.02155
4,BELR833,51.32766,4.36226


As estações usadas neste exemplo (FR04014, BETR801 e London Westminster) são apenas três entradas listadas na tabela de metadados.  
Queremos apenas adicionar as coordenadas dessas três à tabela de medições, cada uma nas linhas correspondentes da air_quality tabela.

In [15]:
air_quality.head()

,date.utc,location,parameter,value
2067,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0
1003,2019-05-07 01:00:00+00:00,FR04014,no2,25.0
100,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5
1098,2019-05-07 01:00:00+00:00,BETR801,no2,50.5
1109,2019-05-07 01:00:00+00:00,London Westminster,pm25,8.0


Usando a função *merge()*, para cada uma das linhas da tabela air_quality, as coordenadas correspondentes são adicionadas air_quality_stations_coord.  
Ambas as tabelas têm a coluna locationem comum, que é usada como chave para combinar as informações.  

Ao escolher a junção *left*, apenas os locais disponíveis na tabela air_quality (à esquerda), ou seja,  
FR04014, BETR801 e London Westminster, são incluídos na tabela resultante.  

A função *merge*, suporta múltiplas opções de junção, semelhantes às operações em estilo de banco de dados.

In [16]:
air_quality = pd.merge(air_quality, stations_coord, how="left", on="location")
air_quality.head()

,date.utc,location,parameter,value,coordinates.latitude,coordinates.longitude
0,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0,51.49467,-0.13193
1,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,48.83724,2.39390
2,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,48.83722,2.39390
3,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5,51.20966,4.43182
4,2019-05-07 01:00:00+00:00,BETR801,no2,50.5,51.20966,4.43182


Adicione a descrição completa e o nome dos parâmetros, fornecidos pela tabela de metadados de parâmetros, à tabela de medições.

In [18]:
air_quality_parameters = pd.read_csv("../data/air_quality_parameters.csv")
air_quality_parameters.head()

,id,description,name
0,bc,Black Carbon,BC
1,co,Carbon Monoxide,CO
2,no2,Nitrogen Dioxide,NO2
3,o3,Ozone,O3
4,pm10,Particulate matter less than 10 micrometers in...,PM10


In [20]:
air_quality = pd.merge(air_quality, air_quality_parameters, how='left', left_on='parameter', right_on='id')
air_quality.head()

,date.utc,location,parameter,value,coordinates.latitude,coordinates.longitude,id_x,description_x,name_x,id_y,description_y,name_y
0,2019-05-07 01:00:00+00:00,London Westminster,no2,23.0,51.49467,-0.13193,no2,Nitrogen Dioxide,NO2,no2,Nitrogen Dioxide,NO2
1,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,48.83724,2.39390,no2,Nitrogen Dioxide,NO2,no2,Nitrogen Dioxide,NO2
2,2019-05-07 01:00:00+00:00,FR04014,no2,25.0,48.83722,2.39390,no2,Nitrogen Dioxide,NO2,no2,Nitrogen Dioxide,NO2
3,2019-05-07 01:00:00+00:00,BETR801,pm25,12.5,51.20966,4.43182,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5,pm25,Particulate matter less than 2.5 micrometers i...,PM2.5
4,2019-05-07 01:00:00+00:00,BETR801,no2,50.5,51.20966,4.43182,no2,Nitrogen Dioxide,NO2,no2,Nitrogen Dioxide,NO2


Em comparação com o exemplo anterior, não há um nome de coluna comum.  
No entanto, a coluna *parameter* na tabela air_quality e a idcoluna na tabela air_quality_parameters_name,  
fornecem a variável medida em um formato comum. 

Os argumentos left_one são usados ​​aqui (em vez de apenas ) para fazer a ligação entre as duas tabelas *.right_onon*

**LEMBRAR**

Várias tabelas podem ser concatenadas tanto em colunas quanto em linhas usando a função *concat*.  
Para mesclar/unir tabelas como em um banco de dados, use a merge função .